In [0]:
from datetime import timedelta, datetime
import math
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, TimestampType
import dbldatagen as dg
import urllib
from pyspark.sql.functions import col

interval = timedelta(days=0, hours=0, seconds=1) #, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
#initial
# start = datetime(2022, 12, 5, 6, 0, 0)
# end = datetime(2023, 1, 5, 12, 0, 0)

start = datetime(2023, 1, 17, 6, 0, 0)
end = datetime(2023, 1, 31, 12, 0, 0)

#datetime(year, month, day, hour=0, minute=0, second=0, microsecond=0, tzinfo=None, *, fold=0)

countiesDF = spark.read.table("default.judete")

counties = list(countiesDF.select("county").toPandas()["county"])
cities = list(countiesDF.select("city").toPandas()["city"])
streets = list(countiesDF.select("street").toPandas()["street"])

coordDf = countiesDF.select("city","lon","lat")

soundSchema = StructType([
    StructField("time", TimestampType(), True),
    StructField("decibels", IntegerType(), True),
    StructField("county", StringType(), True),
    StructField("city", StringType(), True),
    StructField("street", StringType(), True)
])

ds_sound = (dg.DataGenerator(spark, name="sound_traffic_data", rows=20000, partitions=2)
      .withSchema(soundSchema)
      .withColumnSpec("time", "timestamp", begin=start, end=end, interval=interval)
      .withColumnSpec("decibels", minValue=30, maxValue=90, random=True)
      .withColumnSpec("county", values=counties, random=True)
      .withColumnSpec("city", values=cities, random=True)
      .withColumnSpec("street", values=streets, random=True))
      

soundDF = ds_sound.build(withStreaming=True, options={'rowsPerSecond': 10})


(soundDF.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", "/tmp/dbldatagen/soundDatatry/checkpoint_sound")
    .start("/tmp/dbldatagen/soundDatatry/data"))

#%fs rm -r /tmp/dbldatagen

Out[7]: <pyspark.sql.streaming.StreamingQuery at 0x7ff570962f70>